In [28]:
import numpy as np
from scipy.special import gamma, poch, comb, roots_jacobi, jacobi
from scipy.integrate import quadrature, romberg
import functools
import math

In [2]:
@functools.lru_cache(1000)
def I(m, alpha, beta, delta):
  return ((2**(delta+beta+1)) * gamma(delta+1)*gamma(beta+1+m)*gamma(alpha-delta+m)
  /(poch(alpha+1,m)*gamma(beta+delta+2+m)*gamma(alpha-delta) ))

In [3]:
@functools.lru_cache(1000)
def Omega(m, alpha, beta, l):
  if l==0:
    return poch(alpha+1,m)
  elif l>m:
    return 0
  else:
    return poch(m-l+1,l)*poch(alpha+beta+m+1,l)*poch(alpha+l+1,m-l)/ (2**l)

@functools.lru_cache(1000)
def Psi(m, alpha, beta, l):
  if l==0:
    return poch(beta+1,m)
  elif l>m:
    return 0
  else:
    return (-1)**l* poch(m-l+1,l)*poch(alpha+beta+m+1,l)*poch(beta+l+1,m-l) / (2**l)

@functools.lru_cache(1000)
def Delta(m, alpha, beta):

  return (2**(alpha+beta-1)*gamma(m+alpha+1)*gamma(m+beta+1)*((2*m+alpha+beta+2)**2)/
                          (gamma(m+1)*gamma(m+alpha+beta+1)* ((m+1)**2) * ((m+alpha+beta+1)**2)))

In [4]:
@functools.lru_cache(1000)
def type1(m, r, alpha, beta):
  x, _ = roots_jacobi(m,alpha+r+1,beta)

  @functools.lru_cache(1000)
  def e(i):
    if i==r:
      return (-1)**r/math.factorial(r)*I(m, alpha+r+1,beta,alpha+r)
    else:
      sum_term = sum(map(lambda s: e(s)*comb(s,i)*Omega(m, alpha+r+1,beta, s-i), range(i+1,r+1)))
      return (-1)**i/math.factorial(i)*I(m, alpha+r+1,beta,alpha+i)-sum_term/poch(alpha+r+2,m)

  def d(j):
    return (Delta(m, alpha+r+1,beta)*(1+x[j])/(1-x[j])**r/(jacobi(m+1,alpha+r+1,beta)(x[j]))**2)

  return x, np.array(list(map(d, range(m)))), np.array(list(map(lambda i: e(i), range(r+1))))

In [5]:
type1(m=7, r=4, alpha= -1/3 ,beta= 9/5)

(array([-0.89314137, -0.71164856, -0.46606317, -0.1770745 ,  0.13098095,
         0.4324918 ,  0.70399882]),
 array([0.00207763, 0.01930467, 0.07713412, 0.20191162, 0.40414103,
        0.66884323, 0.98217162]),
 array([ 1.56104061e+00, -9.71652396e-02,  3.98143613e-03, -9.37262968e-05,
         9.91762155e-07]))

In [6]:
@functools.lru_cache(1000)
def type2(m, k, alpha, beta):
  y, _ = roots_jacobi(m,alpha,beta+k+1)

  @functools.lru_cache(1000)
  def c(i):
    if i==k:
      return I(m, beta+k+1,alpha, beta+k)/math.factorial(k)
    else:
      sum_term = sum(map(lambda s: c(s)*comb(s,i)*Psi(m, alpha, beta+k+1, s-i), range(i+1,k+1)))
      return I(m, beta+k+1, alpha, beta+i)/math.factorial(i) - sum_term/poch(beta+k+2,m)

  def b(j):
    return (Delta(m, alpha, beta+k+1)*(1-y[j])/(1+y[j])**k/(jacobi(m+1,alpha,beta+k+1)(y[j]))**2)

  return y, np.array(list(map(b, range(m)))), np.array(list(map(lambda i: c(i), range(k+1))))

In [7]:
type2(m=7, k=4, alpha = 3/5, beta= -1/8)

(array([-0.65942958, -0.36080952, -0.0378913 ,  0.28182959,  0.56920852,
         0.79767534,  0.94600231]),
 array([0.43354878, 0.40060633, 0.33520123, 0.24503505, 0.14940954,
        0.0681831 , 0.01643965]),
 array([4.23341973e-01, 3.55565566e-02, 1.72489124e-03, 4.65678271e-05,
        5.56896093e-07]))

In [8]:

def type3(m, k, r, alpha, beta):
  z, _ = roots_jacobi(m,alpha+r+1,beta+k+1)

  @functools.lru_cache(1000)
  def Lambda(m, alpha, beta, l, p):
    if l==0:
      return (2**p)*Psi(m,alpha,beta,0)
    elif l>0:
      return sum(map(lambda j: comb(l,j)*(2**(p-j)) * ((-1)**j) * poch(p-j+1,j) *
                                          Psi(m,alpha,beta,l-j), range(0,l+1)))
  @functools.lru_cache(1000)
  def lamb(i):
    if i==k:
      return I(m, beta+k+1,alpha+r+1, beta+k)/ (2**(r+1)) / math.factorial(k)
    else:
      sum_term = sum(map(lambda s: lamb(s)*comb(s,i)*Lambda(m, alpha+r+1, beta+k+1, s-i, r+1), range(i+1,k+1)))
      return (I(m, beta+k+1, alpha+r+1, beta+i)/ (2**(r+1)) / math.factorial(i) -
              sum_term /poch(beta+k+2,m) / 2**(r+1))

  @functools.lru_cache(1000)
  def Theta(m, alpha, beta, l, p):
    if l==0:
      return (2**p)*Omega(m,alpha,beta,0)
    elif l>0:
      return sum(map(lambda j: comb(l,j)*((2**(p-j))) * poch(p-j+1,j) *
                                          Omega(m,alpha,beta,l-j), range(0,l+1)))

  @functools.lru_cache(1000)
  def mu(i):
    if i==r:
      return ((-1)**r)*I(m, alpha+r+1, beta+k+1, alpha+r)/ (2**(k+1)) / math.factorial(r)
    else:
      sum_term = sum(map(lambda s: mu(s)*comb(s,i)*Theta(m, alpha+r+1, beta+k+1, s-i, k+1), range(i+1,r+1)))
      return (((-1)**i) * I(m, alpha+r+1, beta+k+1, alpha+i)/ (2**(k+1)) / math.factorial(i) - 
              sum_term /poch(alpha+r+2,m) / 2**(k+1))


  def omega(j):
    return (Delta(m, alpha+r+1, beta+k+1)/ (1+z[j])**k / (1-z[j])**r / (jacobi(m+1,alpha+r+1,beta+k+1)(z[j]))**2)

  return (z, np.array(list(map(omega, range(m)))),
          np.array(list(map(lambda i: lamb(i), range(k+1)))),
          np.array(list(map(lambda i: mu(i), range(r+1))))
          )

In [55]:
m=3
k=3
r=2
alpha=0
beta=0
z, omega, lamb, mu = type3(m=m, k=k, r=r, alpha=alpha, beta=beta)
N = 2*m + k + r + 1

In [56]:
p = np.polynomial.Polynomial(np.random.uniform(-1.,1.,N))
def f(x):
  return p(x) * ((1-x)**alpha) * ((1+x)**beta)
lamb @ np.array([p.deriv(i)(-1) for i in range(0,k+1)]) + omega @ np.array([p(zz) for zz in z]) + mu @ np.array([p.deriv(i)(1) for i in range(0,r+1)])

-2.1692996849036983

In [57]:
quadrature(f, -1, 1, maxiter=1000)

(-2.1692996849036987, 8.881784197001252e-16)